# Multi-partite entangelement witness for graph states

Given a graph state, we need to compute the expectation value of all the stabilizer generators and all their products for each edge in the graph

In [1]:
### loading modules
import numpy as np
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

from qutip import *
import cirq

In [2]:
###
#     Functions to construct the qubit graph
###
##-- return the indices of the neighbors of a given qubit
def qubit_neighbor_indices(qubit, qubits):
    
    neighbor_list = list(qubit.neighbors(qubits))
    index_neighbor_list = []
    
    for nn in range(len(neighbor_list)):
        index_neighbor_list.append(qubits.index(neighbor_list[nn]))
    #
    return index_neighbor_list
#

##-- construct the graph of the 2D cluster state under consideration
def build_2D_cluster_state_graph(num_qubits, num_storages):
    qus = cirq.GridQubit.rect(rows = int(num_qubits/num_storages), cols = num_storages)
    
    qubit_graph = nx.Graph()
    qubit_graph.add_nodes_from(range(num_qubits))
    
    for nn in range(num_qubits):
        ind_neighbors = qubit_neighbor_indices(qus[nn], qus)
        
        for ii in ind_neighbors:
            if (nn, ii) not in qubit_graph.edges():
                qubit_graph.add_edge(nn,ii)
            #
        #
    #
    return qubit_graph
#

# def build_2D_cluster_state_graph(num_qubits):
#     qubit_graph = nx.Graph()
#     qubit_graph.add_nodes_from(range(num_qubits))
#     nodes = list(range(num_qubits))
    
#     evens = nodes[::2]
#     odds = nodes[1::2]
    
#     for ii in range(len(evens)):
#         qubit_graph.add_edge(evens[ii], odds[ii])
#     #
    
#     for ii in range(len(evens)-1):
#         qubit_graph.add_edge(evens[ii], evens[ii+1])
#     #
    
#     for ii in range(len(odds)-1):
#         qubit_graph.add_edge(odds[ii], odds[ii+1])
#     #
    
#     return qubit_graph
# #

def build_ring_graph(num_qubits):
    qubit_graph = nx.Graph()
    qubit_graph.add_nodes_from(range(num_qubits))
    nodes = list(range(num_qubits))
    
    evens = nodes[::2]
    odds = nodes[1::2]
    
    for ii in range(len(evens)-1):
        qubit_graph.add_edge(evens[ii], evens[ii+1])
    #
    for ii in range(len(odds)-1):
        qubit_graph.add_edge(odds[ii], odds[ii+1])
    #
    qubit_graph.add_edge(evens[0], odds[0])
    qubit_graph.add_edge(evens[-1], odds[-1])
    
    return qubit_graph
#

def build_tree_graph(nbranches):
    num_qubits = 3*nbranches + 1
    
    qubit_graph = nx.Graph()
    qubit_graph.add_nodes_from(range(num_qubits))
    nodes = list(range(num_qubits))
    
    #- add the edges incident on the root
    for ii in range(nbranches):
        qubit_graph.add_edge(nodes[0], nodes[-2 - ii*3])
    #
    
    for ii in range(nbranches, 0, -1):
        qubit_graph.add_edge(nodes[3*ii - 2], nodes[3*ii - 1])
        qubit_graph.add_edge(nodes[3*ii - 1], nodes[3*ii])
    #
    
    return qubit_graph
#


In [15]:
###
#
#     Some utility functions
#
###

### indices of qubits in a subset defined by an edge
def qubits_in_edge_subset(graph, ee):
    """
    Returns a list with the indices of the qubits in the
    subset defined by the edge ee
    """
    
    neighborsl = list(graph.neighbors(ee[0]))
    neighborsr = list(graph.neighbors(ee[1]))

    all_indexs = sorted(neighborsl + neighborsr)
    return all_indexs
#

### indices of qubits in a subset defined by a node
def qubits_in_node_subset(graph, nn):
    """
    Returns a list with the indices of the qubits in the
    subset defined by the node nn
    """
    
    neighbors = list(graph.neighbors(nn))

    all_indexs = sorted(neighbors + [nn])
    return all_indexs
#

### return the reduced state corresponding to a subset of qubits
def reduced_state_subset(rho, subset):    
    return ptrace(rho, subset)
#

### return the pauli string to be measured given a subset of qubits defined by an edge
def pauli_string_subset_edge(subset, ee):
    pstring = []
    pstring_tag = ""
    for ii in range(len(subset)):
        if subset[ii] in ee:
            pstring.append(sigmay())
            pstring_tag = pstring_tag + "Y" + str(subset[ii])
        else:
            pstring.append(sigmaz())
            pstring_tag = pstring_tag + "Z" + str(subset[ii])
        #
    #
    return (pstring_tag, tensor(pstring))
#

### return the pauli string to be measured given a subset of qubits defined by a node
def pauli_string_subset_node(subset, nn):
    pstring = []
    pstring_tag = ""
    for ii in range(len(subset)):
        if subset[ii] == nn:
            pstring.append(sigmax())
            pstring_tag = pstring_tag + "X" + str(subset[ii])
        else:
            pstring.append(sigmaz())
            pstring_tag = pstring_tag + "Z" + str(subset[ii])
        #
    #
    return (pstring_tag, tensor(pstring))
#

In [16]:
def compute_expectation_generators_on_nodes(graph, rho):
    data_expect = {}
    data_expect["tag"] = []
    data_expect["value"] = []
    
    for nn in list(graph.nodes()):
        nn_subset = qubits_in_node_subset(graph, nn)

        rhor = reduced_state_subset(rho, nn_subset)

        ps_node_tag, ps_node = pauli_string_subset_node(nn_subset, nn)
        
        data_expect["tag"].append(ps_node_tag)
        
        expt = expect(ps_node, rhor)
        
        data_expect["value"].append(expt.real)
        
    #
    return data_expect
#

def compute_expectation_prod_generators_edges(graph, rho):
    edge_list = list(graph.edges())
    
    data_expect = {}
    data_expect["tag"] = []
    data_expect["value"] = []
    
    for ed in edge_list:
        ed_subset = qubits_in_edge_subset(graph, ed)

        rhor = reduced_state_subset(rho, ed_subset)

        ps_edge_tag, ps_edge = pauli_string_subset_edge(ed_subset, ed)
        
        data_expect["tag"].append(ps_edge_tag)
        
        expt = expect(ps_edge, rhor)

        data_expect["value"].append(expt.real)
    #
    return data_expect
#

## The 1D cluster state

In [47]:
### path
pathi = "/Users/munm2002/Documents/projects/graph_states_for_ent_witness/data_states/cluster_state_1D/"
patho = "/Users/munm2002/Documents/projects/graph_states_for_ent_witness/data_expectations_paulis/cluster_state_1D/"
pathg = "/Users/munm2002/Documents/projects/graph_states_for_ent_witness/all_graphs/"

sizes = [4,5,6,7,8,9,10,11,12]

for ss in range(len(sizes)):
    ##-- define the graph
    graph_1d = nx.path_graph(sizes[ss])
    
    #gname = "graph_1D_cluster_%dqubits.txt" % sizes[ss]
    #nx.write_edgelist(graph_1d, pathg+gname)
    
    ##-- load the data
    rho_1d = qload(pathi+"cluster_state_1D_%dqubits.qu" % sizes[ss])
    
    ##-- compute expectation value of the generators
    expt_nodes_1d = compute_expectation_generators_on_nodes(graph_1d, rho_1d)
    expt_nodes_1d_df = pd.DataFrame(expt_nodes_1d)
    
    nname = "expects_on_nodes_cluster_1D_%dqubits.csv" % sizes[ss]
    expt_nodes_1d_df.to_csv(patho+nname, index = False)
    #np.savetxt(patho+nname, expt_nodes_1d)
    
    ##-- compute expectatin value of product of generators on edges
    expt_edges_1d = compute_expectation_prod_generators_edges(graph_1d, rho_1d)
    expt_edges_1d_df = pd.DataFrame(expt_edges_1d)
    
    ename = "expects_on_edges_cluster_1D_%dqubits.csv" % sizes[ss]
    expt_edges_1d_df.to_csv(patho+ename, index = False)
    #np.savetxt(patho+ename, expt_edges_1d)
#
print(pd.DataFrame(expt_nodes_1d))
print(pd.DataFrame(expt_edges_1d))

         tag     value
0       X0Z1  0.921458
1     Z0X1Z2  0.881514
2     Z1X2Z3  0.840810
3     Z2X3Z4  0.840740
4     Z3X4Z5  0.841246
5     Z4X5Z6  0.841500
6     Z5X6Z7  0.841596
7     Z6X7Z8  0.837696
8     Z7X8Z9  0.836645
9    Z8X9Z10  0.839300
10  Z9X10Z11  0.867718
11    Z10X11  0.905864
           tag     value
0       Y0Y1Z2  0.880298
1     Z0Y1Y2Z3  0.842158
2     Z1Y2Y3Z4  0.802989
3     Z2Y3Y4Z5  0.803626
4     Z3Y4Y5Z6  0.803613
5     Z4Y5Y6Z7  0.803894
6     Z5Y6Y7Z8  0.799200
7     Z6Y7Y8Z9  0.800169
8    Z7Y8Y9Z10  0.801734
9   Z8Y9Y10Z11  0.835259
10    Z9Y10Y11  0.858920


## The ring graph state

In [48]:
### path
pathi = "/Users/munm2002/Documents/projects/graph_states_for_ent_witness/data_states/cluster_state_ring/"
patho = "/Users/munm2002/Documents/projects/graph_states_for_ent_witness/data_expectations_paulis/cluster_state_ring/"

sizes = [4,5,6,7,8,9,10,11,12]

for ss in range(len(sizes)):
    ##-- define the graph
    graph_ring = build_ring_graph(sizes[ss])
    
    #gname = "graph_ring_cluster_%dqubits.txt" % sizes[ss]
    #nx.write_edgelist(graph_ring, pathg+gname)
    
    ##-- load the data
    rho_ring = qload(pathi+"cluster_state_ring_%dqubits.qu" % sizes[ss])
    
    ##-- compute expectation value of the generators
    expt_nodes_ring = compute_expectation_generators_on_nodes(graph_ring, rho_ring)
    expt_nodes_ring_df = pd.DataFrame(expt_nodes_ring)
    
    nname = "expects_on_nodes_cluster_ring_%dqubits.csv" % sizes[ss]
    expt_nodes_ring_df.to_csv(patho+nname, index = False)
    #np.savetxt(patho+nname, expt_nodes_ring)
    
    ##-- compute expectatin value of product of generators on edges
    expt_edges_ring = compute_expectation_prod_generators_edges(graph_ring, rho_ring)
    expt_edges_ring_df = pd.DataFrame(expt_edges_ring)
    
    ename = "expects_on_edges_cluster_ring_%dqubits.csv" % sizes[ss]
    expt_edges_ring_df.to_csv(patho+ename, index = False)
    #np.savetxt(patho+ename, expt_edges_ring)

#
print(expt_nodes_ring_df)
print(expt_edges_ring_df)

         tag     value
0     X0Z1Z2  0.855774
1     Z0X1Z3  0.866378
2     Z0X2Z4  0.839399
3     Z1X3Z5  0.811118
4     Z2X4Z6  0.840762
5     Z3X5Z7  0.796104
6     Z4X6Z8  0.841463
7     Z5X7Z9  0.802101
8    Z6X8Z10  0.875740
9    Z7X9Z11  0.794556
10  Z8X10Z11  0.814191
11  Z9Z10X11  0.815773
           tag     value
0     Y0Z1Y2Z4  0.817542
1     Y0Y1Z2Z3  0.796544
2     Z0Y1Y3Z5  0.819784
3     Z0Y2Y4Z6  0.801996
4     Z1Y3Y5Z7  0.771007
5     Z2Y4Y6Z8  0.804115
6     Z3Y5Y7Z9  0.757172
7    Z4Y6Y8Z10  0.836367
8    Z5Y7Y9Z11  0.751097
9   Z6Y8Y10Z11  0.777136
10  Z7Y9Z10Y11  0.761884
11  Z8Z9Y10Y11  0.771419


## The 2D cluster state

In [50]:
### path
pathi = "/Users/munm2002/Documents/projects/graph_states_for_ent_witness/data_states/cluster_state_2D/"
patho = "/Users/munm2002/Documents/projects/graph_states_for_ent_witness/data_expectations_paulis/cluster_state_2D/"

sizes = [4,6,8,10,12]

for ss in range(len(sizes)):
    ##-- define the graph
    graph_2D = build_2D_cluster_state_graph(sizes[ss], 2)
    
    #gname = "graph_2D_cluster_%dqubits_2x%d.txt" % (sizes[ss], sizes[ss]/2)
    #nx.write_edgelist(graph_2D, pathg+gname)
    
    ##-- load the data
    rho_2D = qload(pathi+"cluster_state_2D_%dqubits_2x%d.qu" % (sizes[ss], sizes[ss]/2))
    
    ##-- compute expectation value of the generators
    expt_nodes_2D = compute_expectation_generators_on_nodes(graph_2D, rho_2D)
    expt_nodes_2D_df = pd.DataFrame(expt_nodes_2D)
    
    nname = "expects_on_nodes_cluster_2D_%dqubits_2x%d.csv" % (sizes[ss], sizes[ss]/2)
    expt_nodes_2D_df.to_csv(patho+nname, index = False)
    
    ##-- compute expectatin value of product of generators on edges
    expt_edges_2D = compute_expectation_prod_generators_edges(graph_2D, rho_2D)
    expt_edges_2D_df = pd.DataFrame(expt_edges_2D)
    
    ename = "expects_on_edges_cluster_2D_%dqubits_2x%d.csv" % (sizes[ss], sizes[ss]/2)
    expt_edges_2D_df.to_csv(patho+ename, index = False)
    
#
print(expt_nodes_2D_df)
print(expt_edges_2D_df)

          tag     value
0      X0Z1Z2  0.860567
1      Z0X1Z3  0.838024
2    Z0X2Z3Z4  0.732014
3    Z1Z2X3Z5  0.714720
4    Z2X4Z5Z6  0.694097
5    Z3Z4X5Z7  0.672751
6    Z4X6Z7Z8  0.697630
7    Z5Z6X7Z9  0.656414
8   Z6X8Z9Z10  0.714157
9   Z7Z8X9Z11  0.682697
10   Z8X10Z11  0.755322
11   Z9Z10X11  0.755729
               tag     value
0         Y0Y1Z2Z3  0.771194
1       Y0Z1Y2Z3Z4  0.762375
2       Z0Y1Z2Y3Z5  0.724718
3     Z0Z1Y2Y3Z4Z5  0.680455
4     Z0Y2Z3Y4Z5Z6  0.646451
5     Z1Z2Y3Z4Y5Z7  0.629437
6     Z2Y4Z5Y6Z7Z8  0.623436
7     Z2Z3Y4Y5Z6Z7  0.617065
8     Z3Z4Y5Z6Y7Z9  0.574935
9     Z4Z5Y6Y7Z8Z9  0.606435
10   Z4Y6Z7Y8Z9Z10  0.626114
11   Z5Z6Y7Z8Y9Z11  0.592869
12    Z6Y8Z9Y10Z11  0.699609
13  Z6Z7Y8Y9Z10Z11  0.642474
14    Z7Z8Y9Z10Y11  0.658588
15      Z8Z9Y10Y11  0.764840


In [51]:
### path
pathi = "/Users/munm2002/Documents/projects/graph_states_for_ent_witness/data_states/cluster_state_2D/"

sizes = [9]

for ss in range(len(sizes)):
    ##-- define the graph
    graph_2D = build_2D_cluster_state_graph(sizes[ss], 3)
    
    #gname = "graph_2D_cluster_%dqubits_3x%d.txt" % (sizes[ss], sizes[ss]/3)
    #nx.write_edgelist(graph_2D, pathg+gname)
    
    ##-- load the data
    rho_2D = qload(pathi+"cluster_state_2D_%dqubits_3x%d.qu" % (sizes[ss], sizes[ss]/3))
    
    ##-- compute expectation value of the generators
    expt_nodes_2D = compute_expectation_generators_on_nodes(graph_2D, rho_2D)
    expt_nodes_2D_df = pd.DataFrame(expt_nodes_2D)
    
    nname = "expects_on_nodes_cluster_2D_%dqubits_3x%d.csv" % (sizes[ss], sizes[ss]/3)
    expt_nodes_2D_df.to_csv(patho+nname, index = False)
    
    ##-- compute expectatin value of product of generators on edges
    expt_edges_2D = compute_expectation_prod_generators_edges(graph_2D, rho_2D)
    expt_edges_2D_df = pd.DataFrame(expt_edges_2D)
    
    ename = "expects_on_edges_cluster_2D_%dqubits_3x%d.csv" % (sizes[ss], sizes[ss]/3)
    expt_edges_2D_df.to_csv(patho+ename, index = False)

#
print(expt_nodes_2D_df)
print(expt_edges_2D_df)

          tag     value
0      X0Z1Z3  0.819545
1    Z0X1Z2Z4  0.790209
2      Z1X2Z5  0.849480
3    Z0X3Z4Z6  0.729794
4  Z1Z3X4Z5Z7  0.626787
5    Z2Z4X5Z8  0.758685
6      Z3X6Z7  0.732019
7    Z4Z6X7Z8  0.686376
8      Z5Z7X8  0.770603
               tag     value
0       Y0Y1Z2Z3Z4  0.728184
1       Y0Z1Y3Z4Z6  0.720172
2   Z0Y1Z2Z3Y4Z5Z7  0.662534
3       Z0Y1Y2Z4Z5  0.720780
4       Z1Y2Z4Y5Z8  0.760043
5   Z0Z1Y3Y4Z5Z6Z7  0.614323
6       Z0Y3Z4Y6Z7  0.655503
7   Z1Z3Y4Z5Z6Y7Z8  0.595545
8   Z1Z2Z3Y4Y5Z7Z8  0.608716
9       Z2Z4Y5Z7Y8  0.697347
10      Z3Z4Y6Y7Z8  0.660867
11      Z4Z5Z6Y7Y8  0.660810


## The tree graph state

In [52]:
### path
pathi = "/Users/munm2002/Documents/projects/graph_states_for_ent_witness/data_states/tree_graph_state/"
patho = "/Users/munm2002/Documents/projects/graph_states_for_ent_witness/data_expectations_paulis/tree_graph_state/"

sizes = [1,2,3]

for ss in range(len(sizes)):
    ##-- define the graph
    graph_tree = build_tree_graph(sizes[ss])
    
    #gname = "graph_tree_%dbranches_%dqubits.txt" % (sizes[ss], 3*sizes[ss]+1)
    #nx.write_edgelist(graph_tree, pathg+gname)
    
    ##-- load the data
    rho_tree = qload(pathi+"tree_graph_state_%dbranches_%dqubits.qu" % (sizes[ss], 3*sizes[ss]+1))
    
    ##-- compute expectation value of the generators
    expt_nodes_tree = compute_expectation_generators_on_nodes(graph_tree, rho_tree)
    expt_nodes_tree_df = pd.DataFrame(expt_nodes_tree)
    
    nname = "expects_on_nodes_tree_%dbranches_%dqubits.csv" % (sizes[ss], 3*sizes[ss] + 1)
    expt_nodes_tree_df.to_csv(patho+nname, index = False)
    
    ##-- compute expectatin value of product of generators on edges
    expt_edges_tree = compute_expectation_prod_generators_edges(graph_tree, rho_tree)
    expt_edges_tree_df = pd.DataFrame(expt_edges_tree)
    
    ename = "expects_on_edges_tree_%dbranches_%dqubits.csv" % (sizes[ss], 3*sizes[ss]+1)
    expt_edges_tree_df.to_csv(patho+ename, index = False)
    
#
print(expt_nodes_tree_df)
print(expt_edges_tree_df)

        tag     value
0  X0Z2Z5Z8  0.562909
1      X1Z2  0.890542
2  Z0Z1X2Z3  0.836861
3      Z2X3  0.847978
4      X4Z5  0.887717
5  Z0Z4X5Z6  0.778313
6      Z5X6  0.845732
7      X7Z8  0.852275
8  Z0Z7X8Z9  0.720447
9      Z8X9  0.828335
            tag     value
0  Y0Z2Z5Z7Y8Z9  0.573533
1  Y0Z2Z4Y5Z6Z8  0.582752
2  Y0Z1Y2Z3Z5Z8  0.580088
3      Z0Y1Y2Z3  0.836426
4      Z0Z1Y2Y3  0.842216
5      Z0Y4Y5Z6  0.777908
6      Z0Z4Y5Y6  0.782403
7      Z0Y7Y8Z9  0.720034
8      Z0Z7Y8Y9  0.713337
